1. 在tensorflow一个文本分类的例子上面改的，原网站是：https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/tf2_text_classification.ipynb?pli=1#scrollTo=8wgobMU8fxWW
2. 本模型运行在colab上。

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

In [6]:
import pandas as pd
train_data,test_data=pd.read_csv("train.csv").to_numpy(),pd.read_csv("test_noLabel.csv").to_numpy()
train_data[train_data=="ham"]=0
train_data[train_data=="spam"]=1
test_data[test_data=="ham"]=0
test_data[test_data=="spam"]=1

train_examples, train_labels=train_data[:,2],train_data[:,1]
train_labels = train_labels.astype(np.int64)
test_examples=test_data[:,1]


,ID,Label,Email
0,0,ham,"I don't have anybody's number, I still haven't..."
1,1,spam,Congrats! 2 mobile 3G Videophones R yours. cal...
2,2,ham,She is our sister.. She belongs 2 our family.....
3,3,ham,Ya very nice. . .be ready on thursday
4,4,ham,Okie


In [ ]:
model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples[:3])

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

In [ ]:
x_val = train_examples[:2500]
partial_x_train = train_examples[2500:]

y_val = train_labels[:2500]
partial_y_train = train_labels[2500:]
print(type(partial_x_train))

In [ ]:
history = model.fit(partial_x_train,
          partial_y_train,
          epochs=40,
          batch_size=512,
          validation_data=(x_val, y_val),
          verbose=1)

In [ ]:
res = model.predict(test_examples)
res_labels=[]
for i in range(len(res)):
  if res[i]>0:
    res_labels.append("spam")
  else: 
    res_labels.append("ham")

print(res_labels)

In [ ]:
# 假设test和res都是numpy数组
# 把test的第一列和res合并成一个二维数组
output = np.column_stack((test_data[:, 0], res_labels))
# 使用np.savetxt()函数来输出为csv文件
np.savetxt('output.csv', output, fmt='%s', delimiter=',', header='ID,Label')
